<a href="https://colab.research.google.com/github/stefvas/neural-networks/blob/main/Unsupervised_Learnring_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Εργαστηριακή Άσκηση 2. Μη επιβλεπόμενη μάθηση. 
## Σύστημα συστάσεων βασισμένο στο περιεχόμενο
## Σημασιολογική απεικόνιση δεδομένων με χρήση SOM 

In [66]:
!pip install --upgrade pip
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade nltk
!pip install --upgrade scikit-learn
!pip install --upgrade joblib

##Data insertion

In [67]:
import pandas as pd

dataset_url = "https://drive.google.com/uc?export=download&id=1-tN5rC1QCgwqL67cvIXdMbtd-Xf9Xqow"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

Team seed == 40

In [68]:
import numpy as np

# Στο επόμενη γραμή βάλτε τον αριθμό της ομάδας στο εργαστήριο των νευρωνικών
team_seed_number = 40

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1-oeMf7BFmiH70XObgC9WOTo-a9cDJZwP"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

# επιλέγεται 
my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index] # movie titles (string)
categories = df_data_1.iloc[:, [3]].values[my_index] # movie categories (string)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index] # movie categories in binary form (1 feature per category)
summaries =  df_data_1.iloc[:, [5]].values[my_index] # movie summaries (string)
corpus = summaries[:,0].tolist() # list form of summaries
corpus_df = pd.DataFrame(corpus) # dataframe version of corpus


In [69]:
#print(type(corpus))
punc = '''!@#$%^&*(){}[];:?/.,\|`~_-'''
def simplify_lower(arr):
  corpus_prime = []
  for string in arr:
    temp  = ""
    for char in string:
      if (char.isupper()):
         char = char.lower()
      temp += char
    string = temp
    corpus_prime.append(string)
  return corpus_prime


def simplify_punc(arr):
  corpus_prime = []
  for string in arr:
    temp  = ""
    for char in string:
      if(char in punc):
       count = 0
      else:
        temp += char
    string = temp
    corpus_prime.append(string)
  return corpus_prime

corpus = simplify_lower(corpus)

corpus = simplify_punc(corpus)





In [70]:
ID = 999
print(titles[ID])
print(categories[ID])
print(catbins[ID])
print(corpus[ID])

['Convict 99']
['"Comedy",  "Black-and-white"']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.

# Εφαρμογή 1. Υλοποίηση συστήματος συστάσεων ταινιών βασισμένο στο περιεχόμενο
<img src="http://clture.org/wp-content/uploads/2015/12/Netflix-Streaming-End-of-Year-Posts.jpg" width="70%">

##TFID convertion

In [71]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as stopwords_english
from sklearn.feature_extraction.text import TfidfVectorizer


def tf_idf(mx_df, mn_df, my_stopwrods=list()):

  stopwords_film = ["movie", "film", "plot", "begins", "opens", "starts", "piece", "named", "woman", "women", "man", "men", "prologue", "help", "helping"]
  stopwords = list(stopwords_english) + list(my_stopwrods) + list(stopwords_film)# enhancing stopwords

  vectorizer = TfidfVectorizer(max_df=mx_df, min_df=mn_df, stop_words=stopwords, analyzer='word')
  vectorizer.fit(corpus)
  corpus_tf_idf = vectorizer.transform(corpus)
  print("Dimensions: ", corpus_tf_idf.shape, "\n")


  return corpus_tf_idf

Η συνάρτηση [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) όπως καλείται εδώ **δεν είναι βελτιστοποιημένη**. Οι επιλογές των μεθόδων και παραμέτρων της μπορεί να έχουν **δραματική επίδραση στην ποιότητα των συστάσεων** και είναι διαφορετικές για κάθε dataset. Επίσης, οι επιλογές αυτές έχουν πολύ μεγάλη επίδραση και στη **διαστατικότητα και όγκο των δεδομένων**. Η διαστατικότητα των δεδομένων με τη σειρά της θα έχει πολύ μεγάλη επίδραση στους **χρόνους εκπαίδευσης**, ιδιαίτερα στη δεύτερη εφαρμογή της άσκησης. Ανατρέξτε στα notebooks του εργαστηρίου και στο [FAQ](https://docs.google.com/document/d/1-E4eQkVnTxa3Jb0HL9OAs11bugYRRZ7RNWpu7yh9G4s/edit?usp=sharing) των ασκήσεων.

In [72]:
print(tf_idf(1.0, 0))

Dimensions:  (5000, 52539) 

  (0, 52165)	0.018276263143360158
  (0, 52164)	0.011302700569169849
  (0, 52015)	0.03264021889323426
  (0, 51882)	0.03172604941068616
  (0, 51792)	0.033145149318060466
  (0, 51780)	0.022940377239836447
  (0, 51768)	0.024890518157703857
  (0, 51705)	0.023741574906689406
  (0, 51698)	0.013514636775153157
  (0, 51499)	0.05481237438741874
  (0, 51420)	0.037515940318718785
  (0, 51392)	0.022261513445763933
  (0, 51388)	0.2966426822230622
  (0, 51313)	0.023054645553730675
  (0, 50841)	0.019551640845749688
  (0, 50726)	0.01358319535923186
  (0, 50722)	0.03993165835513382
  (0, 50530)	0.02272027225231764
  (0, 50380)	0.04230475576878638
  (0, 49619)	0.0300563329418967
  (0, 49556)	0.03094744890864895
  (0, 49548)	0.01589422568557242
  (0, 49540)	0.015701187246722453
  (0, 49087)	0.030651378213432518
  (0, 49020)	0.023809610122387907
  :	:
  (4999, 9443)	0.01938413548731317
  (4999, 8978)	0.039876128544933144
  (4999, 8554)	0.02015269020706517
  (4999, 8548)	0.01672

#Suggestion system implementation

Το σύστημα συστάσεων που θα υλοποιήσετε θα είναι μια συνάρτηση content_recommender με τρία ορίσματα: target_movie, max_recommendations και corpus_type. Στην target_movie περνάμε το ID μιας ταινίας-στόχου για την οποία μας ενδιαφέρει να βρούμε παρόμοιες ως προς το περιεχόμενο (τη σύνοψη) ταινίες, max_recommendations στο πλήθος.
Υλοποιήστε τη συνάρτηση ως εξής:

    Για την ταινία-στόχο, θα υπολογίζετε την ομοιότητα συνημιτόνου της με όλες τις ταινίες της συλλογής σας όπως αυτές αναπαριστώνται στο corpus_type.με βάση την ομοιότητα συνημιτόνου που υπολογίσατε, δημιουργήστε ταξινομημένο πίνακα από το μεγαλύτερο στο μικρότερο, με τα indices (ID) των ταινιών.
    
    Για την ταινία-στόχο εκτυπώστε: id, τίτλο, σύνοψη, κατηγορίες (categories)
    
    Για τις max_recommendations ταινίες (πλην της ίδιας της ταινίας-στόχου που έχει cosine similarity 1 με τον εαυτό της) με τη μεγαλύτερη ομοιότητα συνημιτόνου (σε φθίνουσα σειρά), τυπώστε σειρά σύστασης (1 πιο κοντινή, 2 η δεύτερη πιο κοντινή κλπ), ομοιότητα συνημιτόνου, id, τίτλο, σύνοψη, και κατηγορίες (categories)


In [73]:
# function that prints movie info
def print_movie(ID):
  print("ID: ", ID)
  print("Title: ", ', '.join(titles[ID]))
  print("Categories: ", ', '.join(categories[ID]))
  print("Summary:\n", corpus[ID], "\n")

In [74]:
from sklearn.metrics.pairwise import linear_kernel

def content_recommender(ID, max_recommendations, corpus_tf_idf):
  cosine_similarities = linear_kernel(corpus_tf_idf[ID], corpus_tf_idf).flatten()  # calculating cosine similarities using sparse matrices

  cosine_similarities = (-cosine_similarities).argsort()  # sorting the array and saving movies based on ID's

  print_movie(ID)  # print target movie info

  for i in range(1, max_recommendations+1):   # print recommendations
    print("Place:", i)
    print_movie(cosine_similarities[i])

In [75]:
corpus_type = tf_idf(1.0, 0)

Dimensions:  (5000, 52539) 



In [76]:
content_recommender(1, 5, corpus_type)

ID:  1
Title:  Restless
Categories:  "Comedy",  "World cinema",  "Chinese Movies"
Summary:
 leah  is a translator who is traveling international cities in search of romance and closure with her estranged boyfriend jeff she eventually settles in beijing china where she meets and develops a fixation on master sun zhan  who teaches here a chinese game called weiqi when her relationship with master sun zhan takes a turn for the worse leah decides to focus on getting revenge on jeff richard  is a shiftless chineseamerican whose family sends him to beijing with his grandfather's ashes eventually leah and richard's paths cross and a romance develops 

Place: 1
ID:  2737
Title:  Eyes of Fire
Categories:  "Horror"
Summary:
 the film takes place in the year 1750 on the american frontier during the colonial days before the united states declared its independence a group of pioneers narrowly escape persecution when their preacher is accused of adultery and polygamy the preacher will smythe is accu

#Utilization

Αφού υλοποιήσετε τη συνάρτηση content_recommender χρησιμοποιήστε την για να βελτιστοποιήσετε την TfidfVectorizer. Συγκεκριμένα, αρχικά μπορείτε να δείτε τι επιστρέφει το σύστημα για τυχαίες ταινίες-στόχους και για ένα μικρό max_recommendations (2 ή 3). Αν σε κάποιες ταινίες το σύστημα μοιάζει να επιστρέφει σημασιολογικά κοντινές ταινίες σημειώστε το ID τους. Δοκιμάστε στη συνέχεια να βελτιστοποιήσετε την TfidfVectorizer για τα συγκεκριμένα ID ώστε να επιστρέφονται σημασιολογικά κοντινές ταινίες για μεγαλύτερο αριθμό max_recommendations. Παράλληλα, όσο βελτιστοποιείτε την TfidfVectorizer, θα πρέπει να λαμβάνετε καλές συστάσεις για μεγαλύτερο αριθμό τυχαίων ταινιών.

Ταυτόχρονα, μια αντίρροπη κατά κάποιο τρόπο κατεύθυνση της βελτιστοποίησης είναι να χρησιμοποιείτε τις παραμέτρους του TfidfVectorizer έτσι ώστε να μειώνονται οι διαστάσεις του Vector Space Model μέχρι το σημείο που θα αρχίσει να εμφανίζονται επιπτώσεις στην ποιότητα των συστάσεων.

In [77]:
def similar_movies(count):
  movie_list = []

  for i in range(0, 5000): # for all movies
    cnt = count

    cosine_similarities = linear_kernel(corpus_tf_idf[i], corpus_tf_idf).flatten()  # find best matches
    cosine_similarities = (-cosine_similarities).argsort()
    
    categs = categories[i][0]
    categs = categs.split(",")
    
    for category in categs: # find categories similar between th movie and the 3 top matches
      if category.lstrip(' ') in (categories[cosine_similarities[1]][0]) and category.lstrip(' ') in (categories[cosine_similarities[2]][0]) and category.lstrip(' ') in (categories[cosine_similarities[3]][0]):
        cnt = cnt - 1
      if cnt == 0: # if count similar categories are found append the movie to movie_list
        movie_list.append(i)
        break

  return movie_list

we add to stop words list English names to avoid the resemblansh of the movies been based on common names

In [78]:
import urllib.request 
import string

# female english names
target_url = "http://www.cs.cmu.edu/Groups/AI/util/areas/nlp/corpora/names/female.txt"
response = urllib.request.urlopen(target_url).read().decode()
lines_temp = response.split('\n')
names = []

for line in lines_temp:
  if line and line[0] != "#":
    names.append(line.lower())

# male english names
target_url = "http://www.cs.cmu.edu/Groups/AI/util/areas/nlp/corpora/names/male.txt"
response = urllib.request.urlopen(target_url).read().decode()
lines_temp = response.split('\n')

for line in lines_temp:
  if line and line[0] != "#":
    names.append(line.lower())


In [79]:
corpus_tf_idf = tf_idf(1.0, 0, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 49545) 



We see a decrease at the collumns , about 3000

In [80]:
content_recommender(10, 3, corpus_tf_idf)

ID:  10
Title:  Como Esquecer
Categories:  "Drama"
Summary:
 julia  is a professor of english literature who is abandoned by her girlfriend after a relationship that lasted over ten years because of the separation julie moves to rio de janeiro and lives with her best friend hugo  who is gay and lisa  in the new work julia ends up attracting the interest of two students and it is helena  who is after the teacher although she did not feel prepared for a new relationship 

Place: 1
ID:  4711
Title:  Only When I Dance
Categories:  "Documentary"
Summary:
 only when i dance tells the story of two teenagers irlan and isabela who pursue their dreams of becoming professional ballet dancers as a way to escape the violent slums of rio de janeiro 

Place: 2
ID:  3855
Title:  Love, in between
Categories:  "Romance Film",  "Drama"
Summary:
 when the wife of a university professor discovered her husband is having affair with his student she meets his lover they become friends and the relationship bet

#min_df

Ο αποκλεισμός των λιγότερο συχνών λέξεων βοηθά σε κάποιες περιπτώσεις καθώς αποκλείει σπάνιες λέξεις που δεν προσφέρουν στο νόημα, ενώ σε άλλες κάνει το ίδιο αλλά με λέξεις που συσχετίζουν μικρό αριθμό ταινιών της ίδιας κατηγορίας οδηγώντας σε χειρότερα αποτελέσματα.

In [81]:
corpus_tf_idf = tf_idf(1.0, 2, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 23131) 



In this case we succeded better results with that method 

In [82]:
content_recommender(46, 3, corpus_tf_idf)

ID:  46
Title:  Rajakumaran
Categories:  "Drama"
Summary:
 rajakumaran prabhu  is the son of the village chief vijayakumar  yuvaraj  hates rajakumaran and his village selvi  and vaidehi  are in love with their cousin rajakumaran but he chooses vaidehi selvi decides to sacrifice her love her father selvaraj  refuses to give his property to village people as promised and his only request is that rajakumaran gets married with selvi selvi convinced rajakumaran to lie he lies to her father and he gives his property vaidehi committed suicide and rajakumaran reveals his lie one day some goons decide to put a bomb and rajakumaran saves the village the village thinks that the culprit was thangaraj  because he wanted to revenge rajakumaran after his lie angry thangaraj insults rajakumaran's father in public and rajakumaran's father died yuvaraj asks to selvaraj and thangaraj to marry selvi and they accept the village was against the marriage a man  came and said that a deafmute girl who learnt t

In [83]:
corpus_tf_idf = tf_idf(1.0, 15, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 5525) 



In [84]:
content_recommender(46, 3, corpus_tf_idf)

ID:  46
Title:  Rajakumaran
Categories:  "Drama"
Summary:
 rajakumaran prabhu  is the son of the village chief vijayakumar  yuvaraj  hates rajakumaran and his village selvi  and vaidehi  are in love with their cousin rajakumaran but he chooses vaidehi selvi decides to sacrifice her love her father selvaraj  refuses to give his property to village people as promised and his only request is that rajakumaran gets married with selvi selvi convinced rajakumaran to lie he lies to her father and he gives his property vaidehi committed suicide and rajakumaran reveals his lie one day some goons decide to put a bomb and rajakumaran saves the village the village thinks that the culprit was thangaraj  because he wanted to revenge rajakumaran after his lie angry thangaraj insults rajakumaran's father in public and rajakumaran's father died yuvaraj asks to selvaraj and thangaraj to marry selvi and they accept the village was against the marriage a man  came and said that a deafmute girl who learnt t

In this case althouth we see that we don't only get drama recommendations

with continuous trys we conclude that the best result is taken when min-df is 5/6

In [85]:
corpus_tf_idf = tf_idf(1.0, 6, names)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 10640) 



In [86]:
content_recommender(46, 3, corpus_tf_idf)

ID:  46
Title:  Rajakumaran
Categories:  "Drama"
Summary:
 rajakumaran prabhu  is the son of the village chief vijayakumar  yuvaraj  hates rajakumaran and his village selvi  and vaidehi  are in love with their cousin rajakumaran but he chooses vaidehi selvi decides to sacrifice her love her father selvaraj  refuses to give his property to village people as promised and his only request is that rajakumaran gets married with selvi selvi convinced rajakumaran to lie he lies to her father and he gives his property vaidehi committed suicide and rajakumaran reveals his lie one day some goons decide to put a bomb and rajakumaran saves the village the village thinks that the culprit was thangaraj  because he wanted to revenge rajakumaran after his lie angry thangaraj insults rajakumaran's father in public and rajakumaran's father died yuvaraj asks to selvaraj and thangaraj to marry selvi and they accept the village was against the marriage a man  came and said that a deafmute girl who learnt t

Για να επιβεβαιώσουμε την σωστή τιμή του min_df, η οποία επηρεάζει τη διαστατικότητα του πίνακα και άρα και το χρόνο εκπαίδευσης εκτυπώνουμε τα μεγέθη των λιστών που δίνει η συνάρτηση similar_movies που φτιάξαμε, συναρτήσει της min_df, για αριθμό κοινών ειδών μεταξύ των ταινιών 2 και 3.

In [ ]:
import matplotlib.pyplot as plt

length_2 = []
length_3 = []
min_dfs = [1,2,3,4,5,6,7,8,9,10]
for min_df in min_dfs:
  corpus_tf_idf = tf_idf(1.0, min_df, names)
  temp = []
  length_2.append(len(similar_movies(2)))
  length_3.append(len(similar_movies(3)))
  

plt.plot(min_dfs, length_2)
plt.plot(min_dfs, length_3)
plt.xlabel('min_df')
plt.ylabel('Number of Similar Movies')
plt.title('Recommendations Quality to min_df')
plt.legend(['2 same genres', '3 same genres'], loc='center right')
plt.show()

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['luc', 'mar', 'zsa'] not in stop_words.
  % sorted(inconsistent)


Dimensions:  (5000, 49545) 



In [ ]:
print(length_2)
print(length_3)

Παρατηρούμε γενικά μείωση της ποιότητας των επιλογών με την αύξηση του min_df, με μία μικρή αύξηση όμως στις τιμές για count = 2 και min_df μεταξύ 4 και 8. Για count = 3 παρατηρείται μείωση της ποιότητας. Έτσι οι παρατηρήσεις μας επιβεβαιώνονται.

#10 Good recommendations

In [ ]:
corpus_tf_idf = tf_idf(1.0, 6, names)

In [ ]:
content_recommender(12, 5, corpus_tf_idf)

In [ ]:
content_recommender(856, 5, corpus_tf_idf)

In [ ]:
content_recommender(4814, 5, corpus_tf_idf)

In [ ]:
content_recommender(4912, 6, corpus_tf_idf)

In [ ]:
content_recommender(60, 5, corpus_tf_idf)

In [ ]:
content_recommender(455, 4, corpus_tf_idf)

In [ ]:
content_recommender(248, 6, corpus_tf_idf)

In [ ]:
content_recommender(749, 5, corpus_tf_idf)

In [ ]:
content_recommender(1446, 6, corpus_tf_idf)

## Tip: persistence αντικειμένων με joblib.dump


In [ ]:
import joblib

joblib.dump(corpus_tf_idf, 'corpus_tf_idf.pkl', compress=True) 

In [ ]:
!ls -lh

In [ ]:
corpus_tf_idf = joblib.load('corpus_tf_idf.pkl')

# Εφαρμογή 2.  Τοπολογική και σημασιολογική απεικόνιση της ταινιών με χρήση SOM
<img src="https://i.imgur.com/Z4FdurD.jpg" width="60%">

# Δημιουργία dataset

In [ ]:
def build_final_set(doc_limit = 5000, tf_idf_only=False):
    # convert sparse tf_idf to dense tf_idf representation
    dense_tf_idf = corpus_tf_idf.toarray()[0:doc_limit,:]
    if tf_idf_only:
        # use only tf_idf
        final_set = dense_tf_idf
    else:
        # append the binary categories features horizontaly to the (dense) tf_idf features
        final_set = np.hstack((dense_tf_idf, catbins[0:doc_limit,:]))
        # η somoclu θέλει δεδομένα σε float32
    return np.array(final_set, dtype=np.float32)

In [ ]:
final_set = build_final_set()

Τυπώνουμε τις διαστάσεις του τελικού dataset μας. Χωρίς βελτιστοποίηση του TFIDF θα έχουμε περίπου 50.000 χαρακτηριστικά.

In [ ]:
print(corpus_tf_idf.shape)
print(final_set.shape)

Τυπωνουμε τις (μη μηδενικες) τιμες ενος στοιχειου του final_set για να δουμε τι περιεχει. 

In [ ]:
for i in range(len(final_set[0])):
    if final_set[0][i]!=0:
        print(final_set[0][i])

## Εκπαίδευση χάρτη SOM




Θα δουλέψουμε με τη βιβλιοθήκη SOM ["Somoclu"](http://somoclu.readthedocs.io/en/stable/index.html). Εισάγουμε τις somoclu και matplotlib και λέμε στη matplotlib να τυπώνει εντός του notebook (κι όχι σε pop up window).

In [ ]:
# install somoclu
!pip install --upgrade somoclu
# import sompoclu, matplotlib
import somoclu
import matplotlib
# we will plot inside the notebook and not in separate window
%matplotlib inline

Αρχικα οριζουμε τις διαστασεις του SOM για να τις εχουμε ευκαιρες, και οριζουμε το ονομα του αρχειου που ειτε θα αποθηκευσουμε στην συνεχεια η (στους υπολογιστες μας) θα φορτωσουμε για να μην κανουμε την εκπαιδευση του δικτυου.

In [ ]:
n_rows, n_columns = 20,20
som = somoclu.Somoclu(n_columns, n_rows, compactsupport=False)
epochs = 100
filename = 'som' + str(n_rows) + 'x' + str(n_columns) + '@' + str(epochs) + 'e.pkl'

In [ ]:
#don't execute this cell unless the som is already loaded!
#execute cell below to train from scratch the SOM!
som = joblib.load(filename)

In [ ]:
%time som.train(final_set, epochs=epochs)


## Best matching units

In [ ]:
bmus = som.bmus
print(bmus.shape)
print(bmus)
ubmus, indices = np.unique(bmus, return_inverse=True, axis=0)
# το return_inverse επιστρέφει και τους δείκτες indices

In [ ]:
print(ubmus[100:].shape)
print(ubmus[100:])

In [ ]:
print(len(indices))
print(indices)

Κανουμε χρηση της np.where για να δουμε την μορφη εξοδου, αλλα και για να δουμε οτι ολα δουλευουν σωστα.

In [ ]:
print(np.where(indices==0))

## Ομαδοποίηση (clustering)

Τυπικά, η ομαδοποίηση σε ένα χάρτη SOM προκύπτει από το unified distance matrix (U-matrix): για κάθε κόμβο υπολογίζεται η μέση απόστασή του από τους γειτονικούς κόμβους. Εάν χρησιμοποιηθεί μπλε χρώμα στις περιοχές του χάρτη όπου η τιμή αυτή είναι χαμηλή (μικρή απόσταση) και κόκκινο εκεί που η τιμή είναι υψηλή (μεγάλη απόσταση), τότε μπορούμε να πούμε ότι οι μπλε περιοχές αποτελούν clusters και οι κόκκινες αποτελούν σύνορα μεταξύ clusters.
To somoclu δίνει την επιπρόσθετη δυνατότητα να κάνουμε ομαδοποίηση των νευρώνων χρησιμοποιώντας οποιονδήποτε αλγόριθμο ομαδοποίησης του scikit-learn. Στην άσκηση θα χρησιμοποιήσουμε τον k-Means. Για τον αρχικό σας χάρτη δοκιμάστε ένα k=20 ή 25. Οι δύο προσεγγίσεις ομαδοποίησης είναι διαφορετικές, οπότε περιμένουμε τα αποτελέσματα να είναι κοντά αλλά όχι τα ίδια.

Καναμε δοκιμες για τον αριθμο των clusters, και δουλεψαμε με 15, 20, 25 και 30. Βρηκαμε οτι οσο μεγαλωνε το K, τα clusters ηταν πιο διακριτα στις κατηγοριες που περιειχαν (πραγμα εξαιρετικα φυσιολογικο, καθως μικρα Κ τεινουν να μην σεβονται υποκατηγοριες). Ακομα, δεν παρατηρηθηκε καποια αισθητη διαφορα μεταξυ Κ=25 και Κ=30, και επιλεξαμε Κ = 25 καθως θεωρησαμε οτι τα clusters δεν πρεπει να ειναι παρα πολλα, καθως μπορει το νοημα να χαθει και να αναλυουμε λεπτομερειες. 

In [ ]:
from sklearn.cluster import KMeans
n_cluster = 25
algorithm = KMeans(n_clusters=n_cluster)
som.cluster(algorithm=algorithm)


## Αποθήκευση του SOM

Επειδή η αρχικοποίηση του SOM γίνεται τυχαία και το clustering είναι και αυτό στοχαστική διαδικασία, οι θέσεις και οι ετικέτες των νευρώνων και των clusters θα είναι διαφορετικές κάθε φορά που τρέχετε τον χάρτη, ακόμα και με τις ίδιες παραμέτρους. Για να αποθηκεύσετε ένα συγκεκριμένο som και clustering χρησιμοποιήστε και πάλι την `joblib`. Μετά την ανάκληση ενός SOM θυμηθείτε να ακολουθήσετε τη διαδικασία για τα bmus.

In [ ]:
filename = 'som' + str(n_rows) + 'x' + str(n_columns) + '@' + str(epochs) + 'e.pkl'
joblib.dump(som, filename, compress = True)


## Οπτικοποίηση U-matrix, clustering και μέγεθος clusters

Χρησιμοποιουμε τις παραπανω πληροφοριες για να τυπωσουμε το heatmap και τους πινακες ετσι ωστε να εχουμε μια καλυτερη εικονα.

In [ ]:
som.view_umatrix(bestmatches=True, colorbar=True, figsize=(15,15))
print(som.clusters)
_ , no_of_neurons = np.unique(som.clusters, return_counts=True)
cluster_index = np.argsort(no_of_neurons)
print("Cluster Index:")
print(cluster_index)
print("Number of Neurons:")
print(no_of_neurons[cluster_index])


## Σημασιολογική ερμηνεία των clusters

Προκειμένου να μελετήσουμε τις τοπολογικές ιδιότητες του SOM και το αν έχουν ενσωματώσει σημασιολογική πληροφορία για τις ταινίες διαμέσου της διανυσματικής αναπαράστασης με το tf-idf και των κατηγοριών, χρειαζόμαστε ένα κριτήριο ποιοτικής επισκόπησης των clusters. Θα υλοποιήσουμε το εξής κριτήριο: Λαμβάνουμε όρισμα έναν αριθμό (ετικέτα) cluster. Για το cluster αυτό βρίσκουμε όλους τους νευρώνες που του έχουν ανατεθεί από τον k-Means. Για όλους τους νευρώνες αυτούς βρίσκουμε όλες τις ταινίες που τους έχουν ανατεθεί (για τις οποίες αποτελούν bmus). Για όλες αυτές τις ταινίες τυπώνουμε ταξινομημένη τη συνολική στατιστική όλων των ειδών (κατηγοριών) και τις συχνότητές τους. Αν το cluster διαθέτει καλή συνοχή και εξειδίκευση, θα πρέπει κάποιες κατηγορίες να έχουν σαφώς μεγαλύτερη συχνότητα από τις υπόλοιπες. Θα μπορούμε τότε να αναθέσουμε αυτήν/ές την/τις κατηγορία/ες ως ετικέτες κινηματογραφικού είδους στο cluster.

Μπορείτε να υλοποιήσετε τη συνάρτηση αυτή όπως θέλετε. Μια πιθανή διαδικασία θα μπορούσε να είναι η ακόλουθη:

1. Ορίζουμε συνάρτηση `print_categories_stats` που δέχεται ως είσοδο λίστα με ids ταινιών. Δημιουργούμε μια κενή λίστα συνολικών κατηγοριών. Στη συνέχεια, για κάθε ταινία επεξεργαζόμαστε το string `categories` ως εξής: δημιουργούμε μια λίστα διαχωρίζοντας το string κατάλληλα με την `split` και αφαιρούμε τα whitespaces μεταξύ ετικετών με την `strip`. Προσθέτουμε τη λίστα αυτή στη συνολική λίστα κατηγοριών με την `extend`. Τέλος χρησιμοποιούμε πάλι την `np.unique` για να μετρήσουμε συχνότητα μοναδικών ετικετών κατηγοριών και ταξινομούμε με την `np.argsort`. Τυπώνουμε τις κατηγορίες και τις συχνότητες εμφάνισης ταξινομημένα. Χρήσιμες μπορεί να σας φανούν και οι `np.ravel`, `np.nditer`, `np.array2string` και `zip`.

2. Ορίζουμε τη βασική μας συνάρτηση `print_cluster_neurons_movies_report` που δέχεται ως όρισμα τον αριθμό ενός cluster. Με τη χρήση της `np.where` μπορούμε να βρούμε τις συντεταγμένες των bmus που αντιστοιχούν στο cluster και με την `column_stack` να φτιάξουμε έναν πίνακα bmus για το cluster. Προσοχή στη σειρά (στήλη - σειρά) στον πίνακα bmus. Για κάθε bmu αυτού του πίνακα ελέγχουμε αν υπάρχει στον πίνακα μοναδικών bmus που έχουμε υπολογίσει στην αρχή συνολικά και αν ναι προσθέτουμε το αντίστοιχο index του νευρώνα σε μια λίστα. Χρήσιμες μπορεί να είναι και οι `np.rollaxis`, `np.append`, `np.asscalar`. Επίσης πιθανώς να πρέπει να υλοποιήσετε ένα κριτήριο ομοιότητας μεταξύ ενός bmu και ενός μοναδικού bmu από τον αρχικό πίνακα bmus.

3. Υλοποιούμε μια βοηθητική συνάρτηση `neuron_movies_report`. Λαμβάνει ένα σύνολο νευρώνων από την `print_cluster_neurons_movies_report` και μέσω της `indices` φτιάχνει μια λίστα με το σύνολο ταινιών που ανήκουν σε αυτούς τους νευρώνες. Στο τέλος καλεί με αυτή τη λίστα την `print_categories_stats` που τυπώνει τις στατιστικές των κατηγοριών.

Μπορείτε βέβαια να προσθέσετε οποιαδήποτε επιπλέον έξοδο σας βοηθάει. Μια χρήσιμη έξοδος είναι πόσοι νευρώνες ανήκουν στο cluster και σε πόσους και ποιους από αυτούς έχουν ανατεθεί ταινίες.

Θα επιτελούμε τη σημασιολογική ερμηνεία του χάρτη καλώντας την `print_cluster_neurons_movies_report` με τον αριθμός ενός cluster που μας ενδιαφέρει. 

Παράδειγμα εξόδου για ένα cluster (μη βελτιστοποιημένος χάρτης, ωστόσο βλέπετε ότι οι μεγάλες κατηγορίες έχουν σημασιολογική  συνάφεια):

```
Overall Cluster Genres stats:  
[('"Horror"', 86), ('"Science Fiction"', 24), ('"B-movie"', 16), ('"Monster movie"', 10), ('"Creature Film"', 10), ('"Indie"', 9), ('"Zombie Film"', 9), ('"Slasher"', 8), ('"World cinema"', 8), ('"Sci-Fi Horror"', 7), ('"Natural horror films"', 6), ('"Supernatural"', 6), ('"Thriller"', 6), ('"Cult"', 5), ('"Black-and-white"', 5), ('"Japanese Movies"', 4), ('"Short Film"', 3), ('"Drama"', 3), ('"Psychological thriller"', 3), ('"Crime Fiction"', 3), ('"Monster"', 3), ('"Comedy"', 2), ('"Western"', 2), ('"Horror Comedy"', 2), ('"Archaeology"', 2), ('"Alien Film"', 2), ('"Teen"', 2), ('"Mystery"', 2), ('"Adventure"', 2), ('"Comedy film"', 2), ('"Combat Films"', 1), ('"Chinese Movies"', 1), ('"Action/Adventure"', 1), ('"Gothic Film"', 1), ('"Costume drama"', 1), ('"Disaster"', 1), ('"Docudrama"', 1), ('"Film adaptation"', 1), ('"Film noir"', 1), ('"Parody"', 1), ('"Period piece"', 1), ('"Action"', 1)]```

Αρχικα ορισαμε την πρωτη συναρτηση, η οποια παιρνει μια λιστα με ταινιες (list of ints) και τυπωνει τον αριθμο των μοναδικων κατηγοριων (κατηγοριες που εμφανιζονται στην λιστα αυτη με τις ταινιες), καθως και για την καθε κατηγορια τον αριθμο των ταινιων που εχουν σαν κατηγορια αυτην.

In [ ]:
#Imported counter to count unique categories
from collections import Counter
#Imported operator to have a rule of ordering based on the second item of a tuple
import operator

def print_categories_stats(movie_list):
    
    #create the empty list
    total_categories = []
    
    for movie in movie_list:
    
        #take the categories of the string (its a list of one string, so take the first element)
        cats = categories[movie][0]
        #split on comma
        cats = cats.split(',')
    
        for cat in cats:
            #strip of extra spaces
            cat = cat.strip(' ')
            #strip of quotes
            cat = cat.strip('"')
            #put it on the list
            total_categories.append(cat)
    
    #use the counter module to get the unique names of the categories
    uniq = list(Counter(total_categories).keys())
    #as well as the indices of the categories themselves:
    indic = list(Counter(total_categories).values())


    #this didn't work correctly:
    #uniq, indices = np.unique(total_categories, return_index=True)
    
    
    print("Overall Genre Stats:")
    tots = []
    for i in range(len(uniq)):
        tots.append((uniq[i], indic[i]))
    print(len(tots), "Categories.")

    #sort based on the second item of the tuples in tots. 
    tots.sort(key = operator.itemgetter(1), reverse=True)
    print(tots)
    return 0

Για να το τεσταρουμε, χρησιμοποιουμε σαν εισοδο την πληρη λιστα ταινιων (φτιαξαμε μια λιστα με 5000 αριθμους):

In [ ]:
movie_list_ids = list(range(5000))
print_categories_stats(movie_list_ids)

Στην συνεχεια ορισαμε την δευτερη συναρτηση, η οποια παιρνει εναν αριθμο που ειναι το id του cluster και επιστρεφει μια λιστα με αριθμους, που ειναι οι συντεταγμενες των νευρωνων για να βρουμε μεσω των bmus τις ταινιες που θα χρησιμοποιησουμε σαν ορισμα για την πρωτη συναρτηση.

In [ ]:
def print_cluster_neurons_movies_report(cluster_id):

    #use the previously built in variable 
    how_many_neurons = no_of_neurons[cluster_id]

    #use np.where to find where in the cluster map the value of the map is the same as the cluster id
    positions = np.where(som.clusters==cluster_id)
    #make a stack out of the tuples of the coordinates:
    pos = np.column_stack((positions[0], positions[1]))
    
    #create a list of indices to be returned
    INDICES = []
    for position in pos:
        if position in ubmus:
            #make sure the unique index is calculated correctly:
            INDICES.append(int(position[0])*n_rows + int(position[1]))
    return INDICES

Για να ελεγξουμε την ορθη λειτουργια της συναρτησης, την καλουμε με ορισμα 7 (που αντιστοιχει σε ενα cluster που ξερουμε τον αριθμο των νευρωνων που του αντιστοιχουν απο το cluster map):

In [ ]:
print_cluster_neurons_movies_report(7)

Τελος, οριζουμε και την τελευταια συναρτηση, η οποια παιρνει μια λιστα με neuron_ids (list of ints) και καλει την πρωτη συναρτηση ετσι ωστε να τυπωθουν για ολες τις ταινιες που αντιστοιχουν σε αυτους τους νευρωνες τα συνολικα στατιστικα για τις κατηγοριες.

In [ ]:
def neuron_movies_report(neuron_list):
    #First we have to append every movie that is related to every neuron:
    movie_list_IDs = []
    for neuron in neuron_list:

        #We use np.where to find the movies that are related to that neuron
        list_of_movies = list(np.where(indices==neuron))

        #for every movie: append its ID:
        for i in range(len(list_of_movies[0])):
            movie_list_IDs.append(list_of_movies[0][i])

    #now call the first function with the total list to print the stats. 
    print_categories_stats(movie_list_IDs)

Για να ελεγξουμε την συναρτηση αυτη, αλλα και για να δουμε μια συνολικη εικονα, τυπωνουμε εξουθενωτικα για καθε cluster ID τις κατηγοριες με τις οποιες σχετιζεται

In [ ]:
for i in list(cluster_index):
    print("Cluster ID" , i)
    neuron_movies_report(print_cluster_neurons_movies_report(i))